<a href="https://colab.research.google.com/github/rootUJ99/Expensify/blob/master/Copy_of_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Text Classification notebook

In [50]:
%pip install transformers datasets evaluate

In [44]:
from huggingface_hub import notebook_login

notebook_login()

In [31]:
from datasets import load_dataset

dataset = load_dataset("imdb")

In [32]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [33]:
dataset["train"][101]

{'text': 'Assuming this won\'t end up a straight-to-video release, I would have to say void this title at all costs. Unless you\'re bored of good, well-executed movies, that is. I saw this last night at AFI Dallas, and I left with 20 minutes remaining, simply because I didn\'t care anymore (about the plot, not about insulting the director...that is awkward). When you can spot a goof only 5 minutes into the movie (a shot out, shattered window before any shots are fired...and then the window breaks with the first shot), things are going to bad. Let\'s just say this is only an indicator of things to come...unfortunately.<br /><br />I\'ll spare you all the details, but this is sub-par in every manner, even the half-assed acting by Michael Madsen is disappointing when you\'re expecting half-assed acting from him. And the rape scene...Christ! "Shut up and take it" should never be used in a rape scene. EVER.<br /><br />3/10',
 'label': 0}

In [34]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.17.0",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10

In [35]:
def callback_fn(samples):
    return tokenizer(samples['text'], truncation=True)
tokenized_dataset = dataset.map(callback_fn, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [36]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [37]:
import evaluate

accuracy = evaluate.load("accuracy")

In [38]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [39]:
id2label = {0:"NEGATIVE", 1:"POSITIVE"}
label2id = {"NEGATIVE":0, "POSITIVE":1}

In [40]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.17.0",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/

In [41]:
training_args = TrainingArguments(
    output_dir="first_classification_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [42]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/content/first_classification_model is already a clone of https://huggingface.co/rootuj/first_classification_model. Make sure you pull the latest changes with `repo.git_pull()`.


In [16]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25000
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3126


Epoch,Training Loss,Validation Loss,Accuracy
1,0.232300,0.189288,0.927680


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 16
Saving model checkpoint to first_classification_model/checkpoint-1563
Configuration saved in first_classification_model/checkpoint-1563/config.json
Model weights saved in first_classification_model/checkpoint-1563/pytorch_model.bin
tokenizer config file saved in first_classification_model/checkpoint-1563/tokenizer_config.json
Special tokens file saved in first_classification_model/checkpoint-1563/special_tokens_map.json
tokenizer config file saved in first_classification_model/tokenizer_config.json
Special tokens file saved in first_classification_model/special_tokens_map.json


Epoch,Training Loss,Validation Loss,Accuracy
1,0.232300,0.189288,0.927680
2,0.151400,0.233182,0.931800


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 16
Saving model checkpoint to first_classification_model/checkpoint-3126
Configuration saved in first_classification_model/checkpoint-3126/config.json
Model weights saved in first_classification_model/checkpoint-3126/pytorch_model.bin
tokenizer config file saved in first_classification_model/checkpoint-3126/tokenizer_config.json
Special tokens file saved in first_classification_model/checkpoint-3126/special_tokens_map.json
tokenizer config file saved in first_classification_model/tokenizer_config.json
Special tokens file saved in first_classification_model/special_tokens_map.json
Several commits (2) will be pushed upstream.


Training completed. Do not

TrainOutput(global_step=3126, training_loss=0.2054539684752051, metrics={'train_runtime': 3350.0099, 'train_samples_per_second': 14.925, 'train_steps_per_second': 0.933, 'total_flos': 6561288258498624.0, 'train_loss': 0.2054539684752051, 'epoch': 2.0})

inference

In [49]:
trainer.push_to_hub()

Saving model checkpoint to first_classification_model
Configuration saved in first_classification_model/config.json
Model weights saved in first_classification_model/pytorch_model.bin
tokenizer config file saved in first_classification_model/tokenizer_config.json
Special tokens file saved in first_classification_model/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'dataset': {'name': 'imdb', 'type': 'imdb', 'args': 'plain_text'}}


In [51]:
# from transformers import pipeline

# classifier = pipeline("text-classification", model="rootuj/first_classification_model")